In [1]:
TICKER='CNTL'
START='2020-01-01'

In [2]:
import requests
from itertools import chain
import pandas as pd
from datetime import datetime, date, timedelta

from math import pi
from bokeh.plotting import figure,show
from bokeh.models import HoverTool
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [3]:
START = datetime.strptime(START, '%Y-%m-%d')
END = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

In [4]:
def bucket(start_date, end_date, delta = 50):
    left = start_date
    right = left + timedelta(days=delta)
    
    while right < end_date:
        yield (left, right)
        
        left = right + timedelta(days=1)
        right = left + timedelta(days=delta)
    
    yield (left, end_date)

In [5]:
# https://iss.moex.com/iss/engines/stock/markets/shares/boards.xml
# TQBR Т+: Акции и ДР - безадрес.
# iss.json=compact|extended – сокращенный или расширенный формат json;
# iss.version=on|off – получать или нет (по умолчанию - нет) в заголовке.
responses = []
for s,e in bucket(START, END):
    responses.append(requests.get(
        f"http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{TICKER}.json",
             {
                  "iss.meta" : "off",
                  "iss.json" : "compact",
                  "from" : s.strftime('%Y-%m-%d'), 
                  "till" : e.strftime('%Y-%m-%d'),
                  "history.columns": "TRADEDATE,OPEN,CLOSE,LOW,HIGH"
             }).json())
cols = responses[0]['history']['columns']
data = [x for x in chain.from_iterable(map(lambda x: x['history']['data'], responses))]

In [6]:
df = pd.DataFrame(data, columns=cols)
df["TRADEDATE"] = pd.to_datetime(df["TRADEDATE"])

inc = df.CLOSE > df.OPEN
dec = df.OPEN > df.CLOSE
w = 12*60*60*1000 # half day in ms

hover_glyph = HoverTool(mode='vline', line_policy='nearest', names=['marker'],
    tooltips=[
        ('Date', '@date{%F}'),
        ('Price', '@price'),
    ],
    formatters={'@date' : 'datetime'}
)
hover_candle = HoverTool(mode='vline', line_policy='nearest', names=['candle'],
    tooltips=[
        ('Date', '@TRADEDATE{%F}'),
        ('Price', '@CLOSE'),
    ],
    formatters={'@TRADEDATE' : 'datetime'}
)

TOOLS = ["pan","wheel_zoom","box_zoom","reset","save", hover_glyph, hover_candle]

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=800, title = "Stock")

p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df.TRADEDATE, df.HIGH, df.TRADEDATE, df.LOW, color="black")
p.vbar('TRADEDATE', w, 'OPEN', 'CLOSE', fill_color="#D5E1DD", line_color="black", name='candle', source=df[inc])
p.vbar('TRADEDATE', w, 'OPEN', 'CLOSE', fill_color="#F2583E", line_color="black", name='candle', source=df[dec])

def marker(p, df, marker, date, price = None):
    dt = pd.to_datetime(date)
    val = df[df.TRADEDATE == dt]
    
    price = price if price else val.CLOSE
    
    p.segment(dt, df.LOW.min(), dt, val.LOW, color="black", line_dash='dotted')
    p.scatter(dt, df.LOW.min(), marker = marker, size=10, fill_color='blue', line_alpha=1.0,
                  line_cap='butt', line_color='black', 
                  line_dash=[], line_dash_offset=0,
                  line_join='bevel', line_width=1, 
                  name='marker', source={'date':[date], 'price': [price]})

def buy(date, price = None):
    marker(p, df, 'triangle', date, price)

def sell(date, price = None):
    marker(p, df, 'inverted_triangle', date, price)

def info(date, price = None):
    marker(p, df, 'diamond', date, price)

In [12]:
handle = show(p, notebook_handle=True)

In [8]:
info('2020-08-06')

Дата закрытия реестра на получение дивидендов — 10 августа 2020 г. Если вы хотите получить дивиденды, то последний день для покупки бумаг компании — 6 августа. 

In [9]:
info('2020-07-27')

**Алексей Е.:**  
 Акция неликвид  
 Скупают долбаебы  
 Если есть, продавай  
 Это ликвидация компании идёт  
 3.5 млрд это весь капитал  

In [10]:
info('2020-06-29')

11,83 руб. на одну обыкновенную акцию и 11,83 руб. на одну привилегированную рекомендовал выплатить совет директоров Центрального телеграфа в качестве дивидендов за  2019 г.

Всего компания направила на выплату дивидендов 2,499 млрд руб, из них:
– 2,153 млрд руб. из чистой прибыли за 2019 г. 
– 347,170 млн руб. из нераспределенной прибыли общества прошлых лет.